In the entire project, I have divided the given training dataset into train and test splits. 80 percent data is for training and 20 percent for testing. The reason I did this is because I did not have the probabilites for the test dataset so I would not have been able to find accuracy in that case. Also dividing the training dataset into two helps in reducing the total execution time.

I have added comments wherever I felt it was required.

In [ ]:
#Q.1
import os
import torch
import torch.nn as nn
import pandas as pd
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import zipfile

with zipfile.ZipFile("/content/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/images_training_rev1")   #extraction of images


data = pd.read_csv("/content/training_solutions_rev1_updated.csv")  #Reads the CSV file containing galaxy classification labels and probabilities for training images into a Pandas DataFrame.


train_sp, test_sp = train_test_split(data, test_size=0.2, random_state=42) #80% is for training

#already existing resnet-50 is used in this question
model = models.resnet50(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 37)  #Replaces the fully connected layer (model.fc) to output 37 classes (one for each galaxy morphology category).
model.eval()


transform = transforms.Compose([
    transforms.Resize((224, 224)),  #Resizes images to 224x224 (required for ResNet-50).
    transforms.ToTensor(),   #Converts images to PyTorch tensors.
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  #Normalizes pixel values using the ImageNet mean and standard deviation.
])

image_dir = "/content/images_training_rev1/" #here are my extracted images

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #uses GPU if available
model.to(device)  #Moves the model to the selected device.

all_probabilities = []  #an empty list to store probabilites
#tqdm displays a progress bar that updates dynamically as a loop runs.
for galaxy_id in tqdm(test_sp["GalaxyID"]):   #Iterates over the GalaxyID column in the test split (test_sp).
    image_path = os.path.join(image_dir, f"{galaxy_id}.jpg")  #Constructs the image path for each GalaxyID.
    if not os.path.exists(image_path):  #If the image doesn't exist, prints a warning and skips it.
        print(f"Image not found: {image_path}")
        continue

    image = Image.open(image_path).convert("RGB")  #Opens the image and converts it to RGB format.
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():  #Disables gradient calculation for inference (saves memory and computation).
        output = model(image) #Passes the image through the model to get raw output logits.
        probabilities = torch.softmax(output, dim=1).cpu().numpy().flatten()  #Applies the softmax function to convert logits into probabilities for each class.

    all_probabilities.append(probabilities)


class_labels = [f'Class{i+1}' for i in range(37)]  # class_labels: Creates a list of column names (Class1, Class2, ..., Class37).
predictions_df = pd.DataFrame(all_probabilities, columns=class_labels) #Converts all_probabilities to a Pandas DataFrame with class labels as column names.
predictions_df.insert(0, 'GalaxyID', test_sp['GalaxyID'].values) #Inserts the GalaxyID column at the start of the DataFrame.
predictions_df.to_csv("predictions1.csv", index=False)

print("Predictions saved to predictions1.csv")  #predictions are saved in predictions1.csv
#finding accuracy
#here i have converted the probabailites either to 0 or 1. if probability is >0.5, it is converted to 1 and otherwise converted to 0
missing_labels = [label for label in class_labels if label not in test_sp.columns] #Checks if any expected class labels are missing from the test DataFrame.
if missing_labels:
    print(f"Warning: Missing labels {missing_labels}")
else:
    y_true = test_sp[class_labels].values #Extracts ground-truth labels (y_true) for the test split.


    if not np.issubdtype(y_true.dtype, np.bool_): #Ensures ground-truth labels are binary (0 or 1).
        y_true = (y_true > 0).astype(int)
    y_pred = np.array(all_probabilities)

    #i have chosen threshold as 0.5 as it is the mid value of 0 and 1, it can be chosen anything
    threshold = 0.5
    y_pred_binary = (y_pred > threshold).astype(int)  #Converts predicted probabilities (y_pred) into binary predictions using a threshold of 0.5.
    y_true_flat = y_true.ravel()  #converting a multi-dimensional array (or a nested structure) into a single-dimensional array (or a flat structure)
    y_pred_flat = y_pred_binary.ravel()

    accuracy = accuracy_score(y_true_flat, y_pred_flat)  #Accuracy: Fraction of correct predictions.
    f1 = f1_score(y_true_flat, y_pred_flat, average="macro") #F1 Score: Harmonic mean of precision and recall.
    precision = precision_score(y_true_flat, y_pred_flat, average="macro") #Precision: Fraction of relevant predictions among all positive predictions.
    recall = recall_score(y_true_flat, y_pred_flat, average="macro") #Recall: Fraction of true positives among all relevant instances.
    #When using average="macro", the metric is computed as the unweighted mean of the metrics calculated for each class individually. This means each class contributes equally, regardless of its size (number of samples).


    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

100%|██████████| 12316/12316 [50:04<00:00,  4.10it/s]


Predictions saved to predictions1.csv


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4916
F1 Score: 0.3296
Precision: 0.2458
Recall: 0.5000


In [ ]:
#accuracy for Q.1
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

#finding accuracy
#here i have converted the probabailites either to 0 or 1. if probability is >0.5, it is converted to 1 and otherwise converted to 0
missing_labels = [label for label in class_labels if label not in test_sp.columns] #Checks if any expected class labels are missing from the test DataFrame.
if missing_labels:
    print(f"Warning: Missing labels {missing_labels}")
else:
    y_true = test_sp[class_labels].values #Extracts ground-truth labels (y_true) for the test split.


    if not np.issubdtype(y_true.dtype, np.bool_): #Ensures ground-truth labels are binary (0 or 1).
        y_true = (y_true > 0).astype(int)
    y_pred = np.array(all_probabilities)

    #i have chosen threshold as 0.5 as it is the mid value of 0 and 1, it can be chosen anything
    threshold = 0.5
    y_pred_binary = (y_pred > threshold).astype(int)  #Converts predicted probabilities (y_pred) into binary predictions using a threshold of 0.5.
    y_true_flat = y_true.ravel()  #converting a multi-dimensional array (or a nested structure) into a single-dimensional array (or a flat structure)
    y_pred_flat = y_pred_binary.ravel()

    accuracy = accuracy_score(y_true_flat, y_pred_flat)  #Accuracy: Fraction of correct predictions.
    f1 = f1_score(y_true_flat, y_pred_flat, average="macro") #F1 Score: Harmonic mean of precision and recall.
    precision = precision_score(y_true_flat, y_pred_flat, average="macro") #Precision: Fraction of relevant predictions among all positive predictions.
    recall = recall_score(y_true_flat, y_pred_flat, average="macro") #Recall: Fraction of true positives among all relevant instances.
    #When using average="macro", the metric is computed as the unweighted mean of the metrics calculated for each class individually. This means each class contributes equally, regardless of its size (number of samples).


    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")


Accuracy: 0.4916
F1 Score (macro): 0.3296
Precision (macro): 0.2458
Recall (macro): 0.5000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Q.2
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
from PIL import Image
import numpy as np
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import zipfile


with zipfile.ZipFile("/content/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/images_training_rev1")


data = pd.read_csv("/content/training_solutions_rev1_updated.csv")


train_sp, test_sp = train_test_split(data, test_size=0.2, random_state=42)

image_dir = "/content/images_training_rev1/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class GalaxyDataset(Dataset):  #Define a PyTorch Dataset to handle the Galaxy Zoo dataset
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.image_paths = dataframe['GalaxyID']
        self.labels = dataframe.drop(columns=['GalaxyID']).values.astype(np.float32)
        self.image_dir = image_dir
# Initialize the dataset, including the image directory, labels, and optional transformations.
    def __len__(self):
        return len(self.dataframe)
#__len__: Returns the total number of items in the dataset.
    def __getitem__(self, idx):
        img_id = self.image_paths.iloc[idx]
        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx])
        return image, label
#__getitem__: Given an index:
# Load the image using PIL.
# Apply transformations (resize, normalize, etc.).
# Return the image tensor and corresponding label.
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
#creating dataloaders
train_dataset = GalaxyDataset(train_sp, image_dir, transform=transform)
val_dataset = GalaxyDataset(test_sp, image_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
#batch_size: Number of images per batch.
# shuffle: Randomize order for training.
# num_workers: Parallel data loading.
# pin_memory: Optimizes data transfer to GPU.

model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 37)
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()   #The loss function measures how far the model's predictions are from the true labels.
#BCEWithLogitsLoss
# Use: This is used for multi-label binary classification. Each output label is treated as a separate binary classification problem.
# What it does: It applies a sigmoid function to the raw logits (unnormalized output) to squeeze them into the range (0, 1) and then computes the binary cross-entropy loss for each label.

optimizer = optim.Adam(model.parameters(), lr=1e-4) #The optimizer is responsible for updating the model's weights to minimize the loss.
#Adam Optimizer
# Adam (Adaptive Moment Estimation) is an advanced optimization algorithm.
# It combines the benefits of:
# Momentum: Helps accelerate convergence by taking past gradients into account.
# Adaptive Learning Rate: Adjusts the learning rate for each parameter based on the magnitude of past gradients.
# Learning Rate (lr=1e-4): Controls the step size for updating the weights. A smaller learning rate ensures stable convergence but may require more epochs to train.

scaler = GradScaler()  #The Gradient Scaler is used for mixed-precision training, where some computations are performed in lower precision (e.g., float16) to save memory and speed up training.
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True) #Scheduler: Dynamically adjusts the learning rate to ensure efficient and effective training.

def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()


            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
        scheduler.step(running_loss / len(dataloader))
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.sigmoid(outputs).cpu().numpy()
            all_predictions.extend(probabilities)
            all_labels.extend(labels.numpy())

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    predicted_classes = (all_predictions > 0.5).astype(int)  # Threshold for binary prediction
    accuracy = accuracy_score(all_labels.flatten(), predicted_classes.flatten())
    print(f"Accuracy: {accuracy:.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, val_loader)

#this is my generic code for Q.2, since I waited for about 1.5 hours and still could not see any output, i decided to implement on only 500 images in the next code

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-48-e0416add9d93>:82: 

KeyboardInterrupt: 

Since I waited for about 1.5 hours and still could not see any output, i decided to implement the above code on only 500 images in the next code

In [ ]:
#Q.2
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
from PIL import Image
import numpy as np
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import zipfile


with zipfile.ZipFile("/content/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/images_training_rev1")

data = pd.read_csv("/content/training_solutions_rev1_updated.csv")

train_sp, test_sp = data.head(500), data.head(500)


image_dir = "/content/images_training_rev1/"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class GalaxyDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.image_paths = dataframe['GalaxyID']
        self.labels = dataframe.drop(columns=['GalaxyID']).values.astype(np.float32)
        self.image_dir = image_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.image_paths.iloc[idx]
        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx])
        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


train_dataset = GalaxyDataset(train_sp, image_dir, transform=transform)
val_dataset = GalaxyDataset(test_sp, image_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4, pin_memory=True)


model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 37)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = GradScaler()


scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
        scheduler.step(running_loss / len(dataloader))
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.sigmoid(outputs).cpu().numpy()
            all_predictions.extend(probabilities)
            labels_binary = (labels > 0.5).int().cpu().numpy()
            all_labels.extend(labels_binary)

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)

    predicted_classes = (all_predictions > 0.5).astype(int)

    all_labels_flat = all_labels.flatten()
    predicted_classes_flat = predicted_classes.flatten()

    accuracy = accuracy_score(all_labels_flat, predicted_classes_flat)
    print(f"Accuracy: {accuracy*100:.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=3)
evaluate_model(model, val_loader)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-52-9ffff795202f>:81: 

Epoch [1/3], Loss: 0.3135
Epoch [2/3], Loss: 0.3050
Epoch [3/3], Loss: 0.3053
Accuracy: 91.6162


In [28]:
#Q.3
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from PIL import Image
import numpy as np
import pandas as pd
import zipfile
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

with zipfile.ZipFile("/content/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/images_training_rev1")


data = pd.read_csv("/content/training_solutions_rev1_updated.csv")


train_sp, test_sp = data.head(500), data.head(500)  # You can adjust this for a larger dataset

image_dir = "/content/images_training_rev1/images_training_rev1"

# LoRA Layer Implementation
class LoRALinear(nn.Module):  #This defines a new PyTorch module that inherits from nn.Module. The purpose of this module is to modify and extend the behavior of an existing linear layer by introducing low-rank adaptation.
    def __init__(self, original_layer, rank=2):
      # #original_layer: A pre-existing linear layer (e.g., nn.Linear) that this module wraps around and adapts.
      # rank: The rank of the low-rank matrices used in the LoRA approximation. Smaller values for rank reduce the number of trainable parameters, making the adaptation more efficient.
        super(LoRALinear, self).__init__()  #This initializes the base nn.Module class, which is required for any PyTorch module.
        self.original_layer = original_layer  # The original_layer is stored as a part of this module. This layer is used in the forward pass to perform the original computation.
        self.rank = rank  #This defines the size of the low-rank approximation. Instead of using a full weight matrix, LoRA introduces two smaller matrices (lora_A and lora_B) to approximate changes.
        self.lora_A = nn.Parameter(torch.randn(rank, original_layer.in_features) * 0.01)
        self.lora_B = nn.Parameter(torch.randn(original_layer.out_features, rank) * 0.01)

    def forward(self, x):  #The forward method defines how input x is processed through the LoRALinear layer.
        return self.original_layer(x) + (x @ self.lora_A.T @ self.lora_B.T)  #output=original_layer(x)+x⋅Atranspose.Btranspose

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class GalaxyDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        # Construct the full path for each image based on the GalaxyID
        self.image_paths = dataframe['GalaxyID'].apply(lambda x: f"{image_dir}/{x}.jpg")
        self.labels = dataframe.drop(columns=['GalaxyID']).values.astype(np.float32)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.image_paths.iloc[idx]  # Get the full path of the image

        # Check if the image file exists
        if not os.path.exists(img_path):
            print(f"Warning: Image file not found: {img_path}")
            # You could return a default image (e.g., blank or black image) or skip this sample
            image = Image.new("RGB", (224, 224))  # A black placeholder image
        else:
            image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx])
        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


train_dataset = GalaxyDataset(train_sp, transform=transform)
val_dataset = GalaxyDataset(test_sp, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4, pin_memory=True)

# Load ResNet-50 and apply LoRA to the FC layer
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = LoRALinear(nn.Linear(num_features, 37), rank=2)

for name, param in model.named_parameters():
    if 'lora' not in name:
        param.requires_grad = False   #Freezes most pre-trained weights to save computation and focus training on the LoRA parameters.

model = model.to(device)


criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scaler = GradScaler()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

def train_model(model, dataloader, criterion, optimizer, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():  # Mixed precision
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        scheduler.step(running_loss / len(dataloader))
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")


def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.sigmoid(outputs).cpu().numpy()
            all_predictions.extend(probabilities)
            labels_binary = (labels > 0.5).int().cpu().numpy()
            all_labels.extend(labels_binary)

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)

    predicted_classes = (all_predictions > 0.5).astype(int)

    all_labels_flat = all_labels.flatten()
    predicted_classes_flat = predicted_classes.flatten()

    accuracy = accuracy_score(all_labels_flat, predicted_classes_flat)
    print(f"Accuracy: {accuracy*100:.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=3)
evaluate_model(model, val_loader)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

Epoch [1/3], Loss: 0.5454
Epoch [2/3], Loss: 0.3179
Epoch [3/3], Loss: 0.3033
Accuracy: 92.3622


In [30]:
#accuracy for Q.3
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.sigmoid(outputs).cpu().numpy()
            all_predictions.extend(probabilities)
            labels_binary = (labels > 0.5).int().cpu().numpy()
            all_labels.extend(labels_binary)

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    predicted_classes = (all_predictions > 0.5).astype(int)
    accuracy = accuracy_score(all_labels.flatten(), predicted_classes.flatten())
    print(f"Accuracy: {accuracy:.4f}")

evaluate_model(model, val_loader)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy: 0.9236


SUMMARY
ACCURACY FOR THE THREE DIFFERENT QUESTIONS IS AS FOLLOWS:
Q.1)    49.16%
Q.2)    91.62%
Q.3)    92.36%